## Step 1: Hello, Data!

In this step, I load the raw sales transactions CSV file into a DataFrame and display the first three rows to understand the structure of the data.


In [2]:
# ---------- importing the pandas library ----------
import pandas as pd

# ---------- reading  CSV file ----------
file1 = r"D:\Conestoga\Machine Learning Programming\Lab-2\sales_data_with_coupons.csv"
data1 = pd.read_csv(file1)

# ---------- displaying the first 3 rows of the data ----------
print(data1.head(3))



   sno                            Region Country   Item Type Sales Channel  \
0  1.0      middle east and north africa   Libya   Cosmetics       Offline   
1  2.0                     north america  CANADA  Vegetables        Online   
2  3.0    Middle East and North Africa     LIBYA   Baby Food       Offline   

  Order Priority  Order Date     Order ID   Ship Date  Units Sold  Unit Price  \
0              M   18-Oct-14  686800706.0  31-10-2014      8446.0      437.20   
1              M  07-11-2011  185941302.0  2011-12-08      3018.0      154.06   
2              C   31-Oct-16  246222341.0  2016-12-09      1517.0      255.28   

   Unit Cost  Total Revenue  Total Cost  Total Profit coupon_code  
0     263.33     3692591.20  2224085.18    1468506.02      GF24TA  
1      90.93      464953.08   274426.74     190526.34      10AMSP  
2     159.42      387259.76   241840.14     145419.62      TEYPEU  


## Step 2: Pick the Right Container

A `dict` is simple but doesn’t group behavior with data and a `namedtuple` is cleaner than a dict,but no custom functions
A `class` is best here  because it helps organize the data and lets me add my own methods like `clean()` and `total()`.